In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re


#val stands for validation

df = pd.read_csv("./vaccine_train_set.csv")


X1 = df.drop('label', axis=1)
df_x = X1['tweet']  #Keep the tweet
Y = df['label'] #Only keep value
number_of_input = len(df_x)



vectorizer = TfidfVectorizer(strip_accents='ascii', min_df = 2, stop_words='english')   #Vectorize


df_val = pd.read_csv("./vaccine_validation_set.csv")                  #Here just change the name of the file
X1_val = df_val.drop('label', axis=1)
df_x_val = X1_val['tweet']  #Keep the tweet
X_val = vectorizer.fit_transform(df_x_val)


Y_val = df_val['label'] #Only keep value




model = LogisticRegression(multi_class='multinomial',solver ='newton-cg', C = 1 )  #Create the model
 
f = True
it = 1
sets = 10                                       #This is the number of cuts of training set
number = int(it*number_of_input/sets)           #The upper bound of each training set
training_examples = []
score = []
score_val = []


while (f):

  training_examples.append(number)
  input = df_x[0:number] 
                                                #Take the right parts
  output = Y[0:number]
  
  X = vectorizer.transform(input)               #Vectorize
  model.fit(X, output)                          #Train
  #Predict and compute training scores
  Y_predict = model.predict(X)
  precision = precision_score(output, Y_predict, average='weighted')
  recall = recall_score(output, Y_predict, average='weighted')
  f1 = 2*(precision * recall) / (precision + recall)
  score.append(f1)

  #Predict and compute validation scores
  Y_predict = model.predict(X_val)
  precision = precision_score(Y_val, Y_predict, average='weighted')
  recall = recall_score(Y_val, Y_predict, average='weighted')
  f1 = 2*(precision * recall) / (precision + recall)
  score_val.append(f1)


  it = it + 1
  if (number == number_of_input): f = False   #When we check all taining set stop
  number = int(it*number_of_input/sets)       #Take the next upper bound
  if (number > number_of_input): number = number_of_input  #If we pass the limit

print(score)
print(score_val)

#plot
plt.plot(training_examples, score, 'r')
plt.plot(training_examples, score_val, 'g')
plt.title("F1 score")
plt.ylabel('Score')
plt.xlabel('training examples')
plt.show()



plt.show()